In [1]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd 

In [2]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [64]:
def id_split(s):
    s = s + '000'
    s = re.sub('[^\d]', '', s)  
    return s[:4]

print( id_split('5.1-4') )

5140


In [243]:
query = f""" \
    SELECT TOP 5000 
    ID
    , PK_Question
    , QGROUP
    , '@FormName' AS  FormName
    , '@PK_QGroup' AS  FK_QuestionGroup
    , sortpos AS  sortpos
    , IDTEXT AS  IDTEXT
    , NULL AS  Column_Question
    , NULL AS  FK_Question_Picklist
    , ISNULL(FK_PickListType, '')  AS  FK_PickListType
    , ISNULL(FK_QuestionType, '') AS  FK_QuestionType
    , NULL AS  FK_InputType
    , CONVERT(NVARCHAR(5), FK_PickList) AS FK_PickList
    , NULL AS  WarningQuestion
    , NULL AS  reportable
    , NULL AS  ExternalLinkType
    , NULL AS  PK_ExternalLink
    , ISNULL(QText, '') AS  QuestionText
    , ISNULL(help_text, '') AS  help_text 
    FROM vwQuestions  
	WHERE PK_FORM LIKE '2021-A-HVA' AND QGROUP =  2431
"""  
dfq  = get_df(query).fillna('NULL')
cols = list(dfq) # Creates list of all column headers
dfq[cols] = dfq[cols].astype(str)
dfq[10:12]   

,ID,PK_Question,QGROUP,FormName,FK_QuestionGroup,sortpos,IDTEXT,Column_Question,FK_Question_Picklist,FK_PickListType,FK_QuestionType,FK_InputType,FK_PickList,WarningQuestion,reportable,ExternalLinkType,PK_ExternalLink,QuestionText,help_text
10,0-35-6.8.1,22585,2431,@FormName,@PK_QGroup,35,6.8.1,NULL,NULL,0,17,NULL,161,NULL,NULL,NULL,NULL,"If Yes, then is the interdependency upstream, ...",
11,0-36-7,22586,2431,@FormName,@PK_QGroup,36,7,NULL,NULL,0,17,NULL,156,NULL,NULL,NULL,NULL,HVA system management type,


In [270]:
df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='0').fillna('NULL') 
cols = list(df) # Creates list of all column headers
df[cols] = df[cols].astype(str)
df 

,IDTEXT,QPK,SPOS
0,1,28200,0
1,1.1,28201,1
2,1.1.1,28202,2
3,2,28203,3
4,2.1,28204,4
...,...,...,...
118,21.3,28318,118
119,21.4,28319,119
120,21.5,28320,120
121,21.6,28321,121


In [276]:
dff = pd.merge( df, dfq, how='left', on='IDTEXT').fillna('NULL')
dff.sort_values([ 'QPK' ], inplace=True) 
cols = list(dff) # Creates list of all column headers
dff[cols] = dff[cols].astype(str) 
dff

NameError: name 'NULL' is not defined

In [280]:
sql = "" 
for i, r in dff.iterrows():
    s = "(@QID, @FormName, @PK_QGroup, @SORD, N'@IDTEXT', NULL, NULL, NULL, @QTYPE, 1, @PKTYPE, 0, NULL, NULL, NULL, N'@QTEXT','@HTEXT'),"
    s = s.replace('@QID',  r['QPK'])
    s = s.replace('@SORD', str(i)) 
    s = s.replace('@QTYPE', r['FK_QuestionType']) 
    s = s.replace('@PKTYPE', r['FK_PickList']) 
    s = s.replace('@IDTEXT', r['IDTEXT']) 
    s = s.replace('@QTEXT', r['QuestionText'])  
    s = s.replace('@HTEXT', r['help_text'])  
    sql = sql + s + '\r'
sql = sql.replace("'NULL')", "NULL)")    
sql = sql.replace("N'NULL'", "N''")    
with open(fr'd:\temp\update.sql', 'w') as f:
    f.write(sql)  

In [ ]:


query = f""" \
    SELECT TOP 5000 * FROM  vwOrgSubToComponent WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}'
    """  
df_vwOrgSubToComponent = get_df(query)

query = f""" \
    SELECT TOP 5000 * FROM  vwMetricsAnswers WHERE PK_FORM LIKE '%{PK_FORM}%' AND Acronym = '{Acronym}'
    """  
df_vwMetricsAnswers = get_df(query) 

df.to_sql('CVEUnremediated_UPLOAD', engine, index=True, if_exists='replace')

conn = engine.connect() 
conn.execute('DROP TABLE CVEUnremediated_UPLOAD;')
conn.close()